In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler

# โหลดข้อมูลจากไฟล์ CSV
df = pd.read_csv('UNSW_NB15_testing-set.csv')

In [2]:
# Filter 'attack_cat' column for 'Dos' and 'Normal' values
filtered_df = df[df['attack_cat'].isin(['DoS', 'Normal'])]

In [3]:
attack_cat_mapping = {'DoS': 1, 'Normal': 0}
filtered_df['attack_cat_encoded'] = filtered_df['attack_cat'].map(attack_cat_mapping)

C:\Users\ITD\AppData\Local\Temp\ipykernel_14460\1130273881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['attack_cat_encoded'] = filtered_df['attack_cat'].map(attack_cat_mapping)


In [4]:
print(filtered_df[['attack_cat', 'attack_cat_encoded']].head())

  attack_cat  attack_cat_encoded
0     Normal                   0
1     Normal                   0
2     Normal                   0
3     Normal                   0
4     Normal                   0


In [5]:
X = df[['spkts', 'dpkts', 'sbytes', 'dbytes']]
y = filtered_df.loc[:, 'attack_cat_encoded']


In [6]:
from sklearn.utils import resample

# รวม X และ y เข้าด้วยกัน
combined_data = pd.concat([X, y], axis=1)

# หาจำนวนตัวอย่างที่น้อยที่สุด
min_samples = combined_data['attack_cat_encoded'].value_counts().min()

# สร้างข้อมูลที่ undersampled จากแต่ละกลุ่ม
undersampled_data = combined_data.groupby('attack_cat_encoded').apply(lambda x: resample(x, n_samples=min_samples, random_state=42))

# แยก X และ y ออกจาก undersampled_data
X_resampled = undersampled_data[['spkts', 'dpkts', 'sbytes', 'dbytes']]
y_resampled = undersampled_data['attack_cat_encoded']


In [7]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# หา Information Gain ของแต่ละ feature
ig_scores = mutual_info_classif(X_resampled, y_resampled, discrete_features=False)

# แสดง Information Gain ของแต่ละ feature
for feature, ig_score in zip(X_resampled.columns, ig_scores):
    print(f'Information Gain for {feature}: {ig_score}')

# ใช้ DecisionTreeClassifier
model = RandomForestClassifier()

# วนซ้ำโดยใช้ค่า random seed แต่ละครั้ง
for seed in [1, 20, 100, 200, 1000]:
    # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=seed)

    # ใช้ DecisionTreeClassifier
    model.fit(X_train, y_train)

    # ทำนายค่า 'label' ด้วยชุดทดสอบ
    y_pred = model.predict(X_test)

    # ประเมินประสิทธิภาพ
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f'Seed = {seed}, Accuracy: {accuracy}')

    # แสดง Confusion Matrix
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(conf_matrix)

    # แสดง Classification Report
    class_report = metrics.classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)


Information Gain for spkts: 0.22455782178361727
Information Gain for dpkts: 0.33317502573550173
Information Gain for sbytes: 0.5674470766921671
Information Gain for dbytes: 0.42600398494978897
Seed = 1, Accuracy: 0.9828781084386465
Confusion Matrix:
[[2439   43]
 [  41 2383]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      2482
         1.0       0.98      0.98      0.98      2424

    accuracy                           0.98      4906
   macro avg       0.98      0.98      0.98      4906
weighted avg       0.98      0.98      0.98      4906

Seed = 20, Accuracy: 0.9790052996331023
Confusion Matrix:
[[2406   57]
 [  46 2397]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      2463
         1.0       0.98      0.98      0.98      2443

    accuracy                           0.98      4906
   macro avg       0.98      0.98      0.98    